In [1]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-78_1rmka
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-78_1rmka
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 16.0 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp37-cp37m-linux_x86_64.whl size=31526172 sha256=3027f1ab7ac2c256996fac8fab8ab52a4c2e14d436d201fe60e0020d7895425b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ps4_a1j3/wheels/db/2f/07/b84807ee4c9ffc917b90b716977d7fea8f9e841f6173150600
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=a23281a3364a203685db0ffecaa02983a28dc06d39bcb11a81d27610c0c72f73
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built py

In [2]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

from typing import Tuple

# import torch
# import torch.nn.functional as F
# from pytorch3d import _C
# from torch.autograd import Function

from pytorch3d.ops import box3d_overlap
import torch

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun 26 15:31:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 5285
VAL_DATASET_NO = 5050

TRAIN_SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE
VAL_SAMPLES_NO = VAL_DATASET_NO // BATCH_SIZE

TRAIN_DATA_PATH = '/content/drive/MyDrive/mmdetection3d/data/sunrgbd/sunrgbd_trainval/train_data_idx.txt'
VAL_DATA_PATH = '/content/drive/MyDrive/mmdetection3d/data/sunrgbd/sunrgbd_trainval/val_data_idx.txt'

IMAGE_BASE_PATH = '/content/drive/MyDrive/mmdetection3d/data/sunrgbd/sunrgbd_trainval/image'
LABEL_BASE_PATH = '/content/drive/MyDrive/mmdetection3d/data/sunrgbd/sunrgbd_trainval/label_v1'
CALIB_BASE_PATH = '/content/drive/MyDrive/mmdetection3d/data/sunrgbd/sunrgbd_trainval/calib'

In [5]:
def txt_lazy(file_name):
    with open(file_name) as f:

        for line in f.readlines():
            line = line.rstrip('\n')
            line = '0' * (6 - len(line)) + line
            yield line

def batch_generator(DATA_PATH, batch_size):

    file_generator = txt_lazy(DATA_PATH)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            try:   
                file_name = next(file_generator)
            except StopIteration:
                # Reopen generator
                file_generator = txt_lazy(DATA_PATH)

            # file_name = '0' * (6 - len(file_name))
            image_file = file_name
            image_file += '.jpg'
            image_file = os.path.join(IMAGE_BASE_PATH, image_file)

            image = np.asarray(Image.open(image_file).resize((WIDTH, HEIGHT))).flatten()

            batch = np.append(batch, image)

        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(DATA_PATH, batch_size):

    file_generator = txt_lazy(DATA_PATH)

    iteration_no = 0

    while True:
        batch = np.array([])

        iteration_no += 1
        print('Iteration: ' + str(iteration_no))

        for i in range(batch_size):
            try:   
                file_name = next(file_generator)
            except StopIteration:
                # Reopen generator
                file_generator = txt_lazy(DATA_PATH)
                iteration_no = 0

            label_file = os.path.join(LABEL_BASE_PATH, file_name)
            label_file += '.txt'

            label_points = np.array([])

            # calibs points

            calib_file = os.path.join(CALIB_BASE_PATH, file_name)
            calib_file += '.txt'
            
            c_file = open(calib_file, mode = 'r')

            for line in c_file.readlines():
                line = line.split(' ')
                label_points = np.append(label_points, float(line[0]))
                label_points = np.append(label_points, float(line[1]))
                label_points = np.append(label_points, float(line[2]))
                label_points = np.append(label_points, float(line[3]))
                label_points = np.append(label_points, float(line[4]))
                label_points = np.append(label_points, float(line[5]))
                label_points = np.append(label_points, float(line[6]))
                label_points = np.append(label_points, float(line[7]))
                label_points = np.append(label_points, float(line[8]))

            c_file.close()

            # centroid(1), centroid(2), centroid(3), coeffs(1), coeffs(2), coeffs(3), orientation(1), orientation(2));

            l_file = open(label_file, mode = 'r')

            line_ok = 0

            for line in l_file.readlines():
                line = line.split(' ')

                line_ok = 1

                # centroids
                label_points = np.append(label_points, float(line[5]))
                label_points = np.append(label_points, float(line[6]))
                label_points = np.append(label_points, float(line[7]))

                # coeffs
                label_points = np.append(label_points, float(line[8]))
                label_points = np.append(label_points, float(line[9]))
                label_points = np.append(label_points, float(line[10]))

                # orientation
                label_points = np.append(label_points, float(line[11]))
                label_points = np.append(label_points, float(line[12]))

                # Consider only first data
                break

            l_file.close()

            if not line_ok:
                # centroids
                label_points = np.append(label_points, 0)
                label_points = np.append(label_points, 0)
                label_points = np.append(label_points, 0)

                # coeffs
                label_points = np.append(label_points, 0)
                label_points = np.append(label_points, 0)
                label_points = np.append(label_points, 0)

                # orientation
                label_points = np.append(label_points, 0)
                label_points = np.append(label_points, 0)


            batch = np.append(batch, label_points)


        # 26 = 8 label points + 2 * 9 calib points
        yield batch.reshape(batch_size, 26).astype('float')

def model_generator(DATA_PATH, batch_size):
    X_generator = batch_generator(DATA_PATH, batch_size)
    y_generator = label_generator(DATA_PATH, batch_size)

    while True:

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [6]:
def flip_axis_to_camera(pc):
    ''' Flip X-right,Y-forward,Z-up to X-right,Y-down,Z-forward
        Input and output are both (N,3) array
    '''
    pc2 = np.copy(pc)
    pc2[:,[0,1,2]] = pc2[:,[0,2,1]] # cam X,Y,Z = depth X,-Z,Y
    pc2[:,1] *= -1
    return pc2

def flip_axis_to_depth(pc):
    pc2 = np.copy(pc)
    pc2[:,[0,1,2]] = pc2[:,[0,2,1]] # depth X,Y,Z = cam X,Z,-Y
    pc2[:,2] *= -1
    return pc2

class SUNRGBD_Calibration(object):
    ''' Calibration matrices and utils
        We define five coordinate system in SUN RGBD dataset

        camera coodinate:
            Z is forward, Y is downward, X is rightward

        depth coordinate:
            Just change axis order and flip up-down axis from camera coord

        upright depth coordinate: tilted depth coordinate by Rtilt such that Z is gravity direction,
            Z is up-axis, Y is forward, X is right-ward

        upright camera coordinate:
            Just change axis order and flip up-down axis from upright depth coordinate

        image coordinate:
            ----> x-axis (u)
           |
           v
            y-axis (v) 

        depth points are stored in upright depth coordinate.
        labels for 3d box (basis, centroid, size) are in upright depth coordinate.
        2d boxes are in image coordinate

        We generate frustum point cloud and 3d box in upright camera coordinate
    '''

    def __init__(self, Rtilt_arr, K_arr):

        self.Rtilt = np.reshape(Rtilt_arr, (3,3), order='F')
        
        self.K = np.reshape(K_arr, (3,3), order='F')
            
        # K = np.array([float(x) for x in lines[1].split(' ')])
        # self.K = np.reshape(K, (3,3), order='F')
        
        self.f_u = self.K[0,0]
        self.f_v = self.K[1,1]
        self.c_u = self.K[0,2]
        self.c_v = self.K[1,2]
   
    def project_upright_depth_to_camera(self, pc):
        ''' project point cloud from depth coord to camera coordinate
            Input: (N,3) Output: (N,3)
        '''
        # Project upright depth to depth coordinate
        pc2 = np.dot(np.transpose(self.Rtilt), np.transpose(pc[:,0:3])) # (3,n)
        return flip_axis_to_camera(np.transpose(pc2))

    def project_upright_depth_to_image(self, pc):
        ''' Input: (N,3) Output: (N,2) UV and (N,) depth '''
        pc2 = self.project_upright_depth_to_camera(pc)
        uv = np.dot(pc2, np.transpose(self.K)) # (n,3)
        uv[:,0] /= uv[:,2]
        uv[:,1] /= uv[:,2]
        return uv[:,0:2], pc2[:,2]

    def project_upright_depth_to_upright_camera(self, pc):
        return flip_axis_to_camera(pc)

    def project_upright_camera_to_upright_depth(self, pc):
        return flip_axis_to_depth(pc)

    def project_image_to_camera(self, uv_depth):
        n = uv_depth.shape[0]
        x = ((uv_depth[:,0]-self.c_u)*uv_depth[:,2])/self.f_u
        y = ((uv_depth[:,1]-self.c_v)*uv_depth[:,2])/self.f_v
        pts_3d_camera = np.zeros((n,3))
        pts_3d_camera[:,0] = x
        pts_3d_camera[:,1] = y
        pts_3d_camera[:,2] = uv_depth[:,2]
        return pts_3d_camera

    def project_image_to_upright_camerea(self, uv_depth):
        pts_3d_camera = self.project_image_to_camera(uv_depth)
        pts_3d_depth = flip_axis_to_depth(pts_3d_camera)
        pts_3d_upright_depth = np.transpose(np.dot(self.Rtilt, np.transpose(pts_3d_depth)))
        return self.project_upright_depth_to_upright_camera(pts_3d_upright_depth)


In [7]:
class SUNObject3d(object):
    def __init__(self, data):
        self.centroid = np.array([data[0],data[1],data[2]])
        self.unused_dimension = np.array([data[3],data[4],data[5]])
        self.w = data[3]
        self.l = data[4]
        self.h = data[5]
        self.orientation = np.zeros((3,))
        self.orientation[0] = data[6]
        self.orientation[1] = data[7]
        self.heading_angle = -1 * np.arctan2(self.orientation[1], self.orientation[0])


In [8]:
def rotz(t):
    """Rotation about the z-axis."""
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c, -s,  0],
                     [s,  c,  0],
                     [0,  0,  1]])

def compute_box_3d(obj, calib):
    ''' Takes an object and a projection matrix (P) and projects the 3d
        bounding box into the image plane.
        Returns:
            corners_2d: (8,2) array in image coord.
            corners_3d: (8,3) array in in upright depth coord.
    '''
    center = obj.centroid

    # compute rotational matrix around yaw axis
    R = rotz(-1*obj.heading_angle)
    #b,a,c = dimension
    #print R, a,b,c
    
    # 3d bounding box dimensions
    l = obj.l # along heading arrow
    w = obj.w # perpendicular to heading arrow
    h = obj.h

    # rotate and translate 3d bounding box
    x_corners = [-l,l,l,-l,-l,l,l,-l]
    y_corners = [w,w,-w,-w,w,w,-w,-w]
    z_corners = [h,h,h,h,-h,-h,-h,-h]
    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
    corners_3d[0,:] += center[0]
    corners_3d[1,:] += center[1]
    corners_3d[2,:] += center[2]

    # project the 3d bounding box into the image plane
    corners_2d,_ = calib.project_upright_depth_to_image(np.transpose(corners_3d))
    #print 'corners_2d: ', corners_2d
    return corners_2d, np.transpose(corners_3d)

In [9]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

from typing import Tuple

import torch
import torch.nn.functional as F
from pytorch3d import _C
from torch.autograd import Function


# -------------------------------------------------- #
#                  CONSTANTS                         #
# -------------------------------------------------- #
"""
_box_planes and _box_triangles define the 4- and 3-connectivity
of the 8 box corners.
_box_planes gives the quad faces of the 3D box
_box_triangles gives the triangle faces of the 3D box
"""
_box_planes = [
    [0, 1, 2, 3],
    [3, 2, 6, 7],
    [0, 1, 5, 4],
    [0, 3, 7, 4],
    [1, 2, 6, 5],
    [4, 5, 6, 7],
]
_box_triangles = [
    [0, 1, 2],
    [0, 3, 2],
    [4, 5, 6],
    [4, 6, 7],
    [1, 5, 6],
    [1, 6, 2],
    [0, 4, 7],
    [0, 7, 3],
    [3, 2, 6],
    [3, 6, 7],
    [0, 1, 5],
    [0, 4, 5],
]


def _check_coplanar(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    faces = torch.tensor(_box_planes, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    P, V = faces.shape
    # (B, P, 4, 3) -> (B, P, 3)
    v0, v1, v2, v3 = verts.reshape(B, P, V, 3).unbind(2)

    # Compute the normal
    e0 = F.normalize(v1 - v0, dim=-1)
    e1 = F.normalize(v2 - v0, dim=-1)
    normal = F.normalize(torch.cross(e0, e1, dim=-1), dim=-1)

    # Check the fourth vertex is also on the same plane
    mat1 = (v3 - v0).view(B, 1, -1)  # (B, 1, P*3)
    mat2 = normal.view(B, -1, 1)  # (B, P*3, 1)
    if not (mat1.bmm(mat2).abs() < eps).all().item():
        msg = "Plane vertices are not coplanar"
        raise ValueError(msg)

    return


def _check_nonzero(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    """
    Checks that the sides of the box have a non zero area
    """
    faces = torch.tensor(_box_triangles, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    T, V = faces.shape
    # (B, T, 3, 3) -> (B, T, 3)
    v0, v1, v2 = verts.reshape(B, T, V, 3).unbind(2)

    normals = torch.cross(v1 - v0, v2 - v0, dim=-1)  # (B, T, 3)
    face_areas = normals.norm(dim=-1) / 2

    if (face_areas < eps).any().item():
        msg = "Planes have zero areas"
        raise ValueError(msg)

    return


class _box3d_overlap(Function):
    """
    Torch autograd Function wrapper for box3d_overlap C++/CUDA implementations.
    Backward is not supported.
    """

    @staticmethod
    def forward(ctx, boxes1, boxes2):
        """
        Arguments defintions the same as in the box3d_overlap function
        """
        vol, iou = _C.iou_box3d(boxes1, boxes2)
        return vol, iou

    @staticmethod
    def backward(ctx, grad_vol, grad_iou):
        raise ValueError("box3d_overlap backward is not supported")


def my_box3d_overlap(
    boxes1: torch.Tensor, boxes2: torch.Tensor, eps: float = 1e-4
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes the intersection of 3D boxes1 and boxes2.

    Inputs boxes1, boxes2 are tensors of shape (B, 8, 3)
    (where B doesn't have to be the same for boxes1 and boxes1),
    containing the 8 corners of the boxes, as follows:

        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)


    NOTE: Throughout this implementation, we assume that boxes
    are defined by their 8 corners exactly in the order specified in the
    diagram above for the function to give correct results. In addition
    the vertices on each plane must be coplanar.
    As an alternative to the diagram, this is a unit bounding
    box which has the correct vertex ordering:

    box_corner_vertices = [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]

    Args:
        boxes1: tensor of shape (N, 8, 3) of the coordinates of the 1st boxes
        boxes2: tensor of shape (M, 8, 3) of the coordinates of the 2nd boxes
    Returns:
        vol: (N, M) tensor of the volume of the intersecting convex shapes
        iou: (N, M) tensor of the intersection over union which is
            defined as: `iou = vol / (vol1 + vol2 - vol)`
    """
    if not all((8, 3) == box.shape[1:] for box in [boxes1, boxes2]):
        raise ValueError("Each box in the batch must be of shape (8, 3)")

    # pyre-fixme[16]: `_box3d_overlap` has no attribute `apply`.
    vol, iou = _box3d_overlap.apply(boxes1, boxes2)

    if iou == np.nan:
        print('Nan detected')
        iou = 0

    return vol, iou


In [10]:
# 3D IoU Custom Metric

def iou_3D(y_true, y_pred):

    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    # Extract calib from y_true
    Rtilt_arr = y_true[:, 0:9]
    K_arr = y_true[:, 9:18]

    calib_obj = []
    for i in range(y_true.shape[0]):
        calib_obj.append(SUNRGBD_Calibration(Rtilt_arr[i], K_arr[i]))

    y_true_data = y_true[:, 18:]

    y_true_obj = []
    y_pred_obj = []

    for i in range(y_true.shape[0]):
        y_true_obj.append(SUNObject3d(y_true[i, 18:]))

    for i in range(y_pred.shape[0]):
        y_pred_obj.append(SUNObject3d(y_pred[i]))
    
    y_true_precompute = np.array([])
    y_pred_precompute = np.array([])

    for i in range(y_true.shape[0]):
      y_true_corners_2d, y_true_corners_3d = compute_box_3d(y_true_obj[i], calib_obj[i])
      y_pred_corners_2d, y_pred_corners_3d = compute_box_3d(y_pred_obj[i], calib_obj[i])

      y_true_precompute = np.append(y_true_precompute, y_true_corners_3d)
      y_pred_precompute = np.append(y_pred_precompute, y_pred_corners_3d)

    # Convert form tf tensor to numpy
    # y_true_np = y_true_precompute.reshape(-1, 8, 2).astype(np.float32)
    # y_pred_np = y_pred_precompute.reshape(-1, 8, 2).astype(np.float32)
    y_true_np = y_true_precompute.reshape(-1, 8, 3).astype(np.float32)
    y_pred_np = y_pred_precompute.reshape(-1, 8, 3).astype(np.float32)

    # Add the fictive third dimension
    # y_true_np = np.append(y_true_np, np.zeros((y_true_np.shape[0], y_true_np.shape[1], 1)), axis=2)
    # y_pred_np = np.append(y_pred_np, np.zeros((y_pred_np.shape[0], y_pred_np.shape[1], 1)), axis=2)


    # Reorder points to be compatible with pytorch3D requirements
    y_true_np_reorder = np.zeros(y_true_np.shape)
    y_pred_np_reorder = np.zeros(y_pred_np.shape)
    y_true_np_reorder[:, 0, :] = y_true_np[:, 2, :]
    y_pred_np_reorder[:, 0, :] = y_pred_np[:, 2, :]
    y_true_np_reorder[:, 1, :] = y_true_np[:, 3, :]
    y_pred_np_reorder[:, 1, :] = y_pred_np[:, 3, :]
    y_true_np_reorder[:, 2, :] = y_true_np[:, 7, :]
    y_pred_np_reorder[:, 2, :] = y_pred_np[:, 7, :]
    y_true_np_reorder[:, 3, :] = y_true_np[:, 6, :]
    y_pred_np_reorder[:, 3, :] = y_pred_np[:, 6, :]
    y_true_np_reorder[:, 4, :] = y_true_np[:, 1, :]
    y_pred_np_reorder[:, 4, :] = y_pred_np[:, 1, :]
    y_true_np_reorder[:, 5, :] = y_true_np[:, 0, :]
    y_pred_np_reorder[:, 5, :] = y_pred_np[:, 0, :]
    y_true_np_reorder[:, 6, :] = y_true_np[:, 4, :]
    y_pred_np_reorder[:, 6, :] = y_pred_np[:, 4, :]
    y_true_np_reorder[:, 7, :] = y_true_np[:, 5, :]
    y_pred_np_reorder[:, 7, :] = y_pred_np[:, 5, :]
    ''' 
            1 -------- 0
           /|         /|
          2 -------- 3 .
          | |        | |
          . 5 -------- 4
          |/         |/
          6 -------- 7
                |
                V
        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)
    '''

    # Add the 1 value for "back" points
    # y_true_np_reorder[:, 4:, :] = 1
    # y_pred_np_reorder[:, 4:, :] = 1

    # Scale the value back to original dimensions
    # y_true_np_reorder[:, :, 0] *= WIDTH
    # y_true_np_reorder[:, :, 1] *= HEIGHT
    # y_pred_np_reorder[:, :, 0] *= WIDTH
    # y_pred_np_reorder[:, :, 1] *= HEIGHT

    # Convert from numpy to tf tensor
    y_true_torch_init = torch.tensor(y_true_np_reorder)
    y_pred_torch_init = torch.tensor(y_pred_np_reorder)

    # Reshape to (None, 8, 3) dimension
    y_true_torch_reshaped = torch.reshape(y_true_torch_init, (-1, 8, 3)).type(torch.float32)
    y_pred_torch_reshaped = torch.reshape(y_pred_torch_init, (-1, 8, 3)).type(torch.float32)


    # _, iou_3d = box3d_overal(torch_y_true, torch_y_pred)
    # _, iou_3d = my_box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped)
    try:
        _, iou_3d = box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped)
    except ValueError:
        print('Got ValueError')
        iou_3d = 0

    # _, iou_3d = my_box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped, 0.0)

    return iou_3d
    # return 0


In [11]:
def custom_mse(y_true, y_pred):
    y_true = y_true.numpy().reshape((-1, 26))[:, 18:]
    y_true = tf.convert_to_tensor(y_true, np.float32)

    return tf.math.squared_difference(y_pred, y_true)

In [12]:
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(8)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss=custom_mse, optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

9420800/9406464 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [13]:
model.fit(x = model_generator(TRAIN_DATA_PATH, BATCH_SIZE),
          steps_per_epoch = TRAIN_SAMPLES_NO,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Got ValueError
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Ite

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


Streaming output truncated to the last 5000 lines.
Iteration: 388
Iteration: 389
Iteration: 390
Iteration: 391
Iteration: 392
Iteration: 393
Iteration: 394
Iteration: 395
Iteration: 396
Iteration: 397
Iteration: 398
Iteration: 399
Iteration: 400
Iteration: 401
Iteration: 402
Iteration: 403
Iteration: 404
Iteration: 405
Iteration: 406
Iteration: 407
Iteration: 408
Iteration: 409
Iteration: 410
Iteration: 411
Iteration: 412
Iteration: 413
Iteration: 414
Iteration: 415
Iteration: 416
Iteration: 417
Iteration: 418
Iteration: 419
Iteration: 420
Iteration: 421
Got ValueError
Iteration: 422
Iteration: 423
Iteration: 424
Iteration: 425
Iteration: 426
Iteration: 427
Iteration: 428
Iteration: 429
Iteration: 430
Iteration: 431
Iteration: 432
Iteration: 433
Iteration: 434
Iteration: 435
Iteration: 436
Iteration: 437
Iteration: 438
Iteration: 439
Iteration: 440
Iteration: 441
Iteration: 442
Iteration: 443
Iteration: 444
Iteration: 445
Iteration: 446
Iteration: 447
Iteration: 448
Iteration: 449
Iter

In [14]:
model.evaluate(
    x = model_generator(VAL_DATA_PATH, BATCH_SIZE),
    verbose = 2,
    steps = VAL_SAMPLES_NO
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


Got ValueError
Iteration: 164
Iteration: 165
Iteration: 166
Iteration: 167
Iteration: 168
Iteration: 169
Got ValueError
Iteration: 170
Iteration: 171
Iteration: 172
Iteration: 173
Iteration: 174
Iteration: 175
Iteration: 176
Iteration: 177
Iteration: 178
Iteration: 179
Iteration: 180
Iteration: 181
Iteration: 182
Iteration: 183
Iteration: 184
Iteration: 185
Iteration: 186
Iteration: 187
Iteration: 188
Iteration: 189
Iteration: 190
Iteration: 191
Iteration: 192
Iteration: 193
Iteration: 194
Iteration: 195
Iteration: 196
Iteration: 197
Iteration: 198
Iteration: 199
Iteration: 200
Iteration: 201
Iteration: 202
Iteration: 203
Iteration: 204
Iteration: 205
Iteration: 206
Iteration: 207
Iteration: 208
Iteration: 209
Iteration: 210
Iteration: 211
Iteration: 212
Iteration: 213
Iteration: 214
Iteration: 215
Iteration: 216
Iteration: 217
Iteration: 218
Iteration: 219
Iteration: 220
Iteration: 221
Iteration: 222
Iteration: 223
Iteration: 224
Iteration: 225
Iteration: 226
Iteration: 227
Iteration:

[0.31253138184547424, 0.053366292268037796]

In [ ]:
model.predict(
    x = batch_generator(VAL_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 1s - 558ms/epoch - 558ms/step


array([[ 0.28731447,  2.087208  , -0.84225655,  0.42452714,  0.26025033,
         0.56873536, -0.05480846, -0.3447894 ],
       [-0.03213147,  2.4031034 , -0.82628715,  0.56564385,  0.3800609 ,
         0.49034733, -0.01056157, -0.5467577 ],
       [-0.12947771,  2.4718342 , -0.8471728 ,  0.49169022,  0.39792502,
         0.40972322,  0.01283253, -0.61854184],
       [-0.12077872,  2.3281999 , -0.6927819 ,  0.5805964 ,  0.4262529 ,
         0.46391898, -0.3550171 , -0.6249535 ],
       [-0.25078952,  2.0623422 , -0.8774142 ,  0.53577876,  0.36143282,
         0.4794299 ,  0.1698384 , -0.36104983],
       [-0.5462662 ,  2.3547783 , -0.88618016,  0.6396667 ,  0.506896  ,
         0.5435529 , -0.07592407, -0.34681648],
       [ 0.21436496,  2.437717  , -0.72574157,  0.5516932 ,  0.3852414 ,
         0.49680725, -0.5364397 , -0.48911792],
       [-0.43736675,  2.4991777 , -0.7776916 ,  0.7327217 ,  0.39473373,
         0.5166151 ,  0.18741558, -0.73429406],
       [-0.0943569 ,  2.0676262 